In [1]:
import pyspark
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType, StringType, FloatType, TimestampType, StructType, StructField


data = spark.read.json("s3://big-datasets-over-gb/iot-devices/iot_devices.json")

data.tail(2)

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6,application_1651428963520_0008,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(battery_level=4, c02_level=1358, cca2='DE', cca3='DEU', cn='Germany', device_id=198163, device_name='meter-gauge-198163jphYrUReMD', humidity=56, ip='93.135.125.41', latitude=48.19, lcd='yellow', longitude=11.38, scale='Celsius', temp=30, timestamp=1458444061098), Row(battery_level=0, c02_level=1291, cca2='PL', cca3='POL', cn='Poland', device_id=198164, device_name='sensor-pad-198164Ndlyvs0r9C', humidity=98, ip='80.55.20.25', latitude=53.08, lcd='yellow', longitude=18.62, scale='Celsius', temp=12, timestamp=1458444061098)]

In [58]:
data.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- battery_level: long (nullable = true)
 |-- c02_level: long (nullable = true)
 |-- cca2: string (nullable = true)
 |-- cca3: string (nullable = true)
 |-- cn: string (nullable = true)
 |-- device_id: long (nullable = true)
 |-- device_name: string (nullable = true)
 |-- humidity: long (nullable = true)
 |-- ip: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- lcd: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- scale: string (nullable = true)
 |-- temp: long (nullable = true)
 |-- timestamp: long (nullable = true)

In [125]:
df = data.\
select([F.col('device_id').alias('id'),\
             'timestamp','battery_level',\
             'c02_level',F.col('cn').alias('country'),\
             'device_name', 'humidity','temp'])

#lets cache the dataset
df.cache()
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+-------------+-------------+---------+-------------+--------------------+--------+----+
| id|    timestamp|battery_level|c02_level|      country|         device_name|humidity|temp|
+---+-------------+-------------+---------+-------------+--------------------+--------+----+
|  1|1458444054093|            8|      868|United States|meter-gauge-1xbYRYcj|      51|  34|
|  2|1458444054119|            7|     1473|       Norway|   sensor-pad-2n2Pea|      70|  11|
|  3|1458444054120|            2|     1556|        Italy| device-mac-36TWSKiT|      44|  19|
|  4|1458444054121|            6|     1080|United States|   sensor-pad-4mzWkz|      32|  28|
|  5|1458444054122|            4|      931|  Philippines|therm-stick-5gimp...|      62|  25|
|  6|1458444054122|            3|     1210|United States|sensor-pad-6al7RT...|      51|  27|
|  7|1458444054123|            3|     1129|        China|meter-gauge-7GeDoanM|      26|  18|
|  8|1458444054123|            0|     1536|        Japan|sensor-pad-8x

##### show distinct device names. use regex to remove anything after hyphen which starts with a number

In [115]:
regex = '-[0-9e]\w+'

df = df.withColumn('device_name', F.regexp_replace(F.col('device_name'), regex, ""))
df.select('device_name').distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+
|device_name|
+-----------+
|meter-gauge|
| device-mac|
|therm-stick|
| sensor-pad|
+-----------+

##### first 10 devices with temperatures excedding 30 and humidity greater than 70 in UK,USA or UAE, sorted by C02 and battery level 

In [65]:


df.where(F.col('country').contains('United')).select('country').distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|             country|
+--------------------+
|       United States|
|United Arab Emirates|
|      United Kingdom|
+--------------------+

In [116]:
df.where(F.col('country').\
         contains('United')).\
filter("""temp > 30 and humidity > 70""").\
sort(F.col("C02_level").desc(), F.col("battery_level").desc()).\
limit(10).\
show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------------+-------------+---------+--------------+-----------+--------+--------------+----+
|    id|    timestamp|battery_level|c02_level|       country|device_name|humidity|            ip|temp|
+------+-------------+-------------+---------+--------------+-----------+--------+--------------+----+
|  4872|1458444055015|            7|     1599| United States| sensor-pad|      96|64.125.193.177|  34|
| 28711|1458444056305|            6|     1599|United Kingdom|meter-gauge|      95| 212.113.11.17|  31|
| 88359|1458444057734|            8|     1598| United States| device-mac|      93|  66.250.15.54|  33|
|141720|1458444059708|            6|     1598|United Kingdom| sensor-pad|      98|   80.192.74.1|  32|
| 35172|1458444056451|            5|     1598| United States| sensor-pad|      85|216.49.204.165|  31|
| 14782|1458444055958|            4|     1598| United States| sensor-pad|      94|  129.105.47.1|  34|
| 32336|1458444056392|            3|     1598| United States| sensor-pad|

##### Compute the average humidity and temperature of all the devices in each country whose temperature and humidity are greater than 25 and 75 resepctively

In [83]:
sorted_df = df.filter("temp > 25 and humidity > 75")\
.select("temp", "humidity", "country")\
.groupBy("country")\
.avg()\
.sort(F.col("avg(temp)").desc(), F.col("avg(humidity)").desc())\

sorted_df.show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------+------------------+-----------------+
|country                    |avg(temp)         |avg(humidity)    |
+---------------------------+------------------+-----------------+
|Monaco                     |34.0              |91.0             |
|Anguilla                   |34.0              |83.0             |
|British Virgin Islands     |34.0              |81.0             |
|Turkmenistan               |34.0              |80.0             |
|Suriname                   |34.0              |79.0             |
|Gibraltar                  |34.0              |78.0             |
|Liechtenstein              |34.0              |76.0             |
|Vanuatu                    |33.5              |84.0             |
|Cameroon                   |33.0              |91.0             |
|Fiji                       |33.0              |78.0             |
|Haiti                      |32.5              |89.0             |
|Brunei                     |32.5              |89.0          

##### Number of the devices per country

In [92]:
df.select("country", "id").groupBy("country").count().distinct().orderBy(F.col("count").desc()).limit(100).show()
            

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-----+
|          country|count|
+-----------------+-----+
|    United States|68545|
|            China|14455|
|            Japan|12100|
|Republic of Korea|11879|
|          Germany| 7942|
|   United Kingdom| 6486|
|           Canada| 6041|
|           Russia| 5989|
|           France| 5305|
|           Brazil| 3224|
|        Australia| 3119|
|            Italy| 2915|
|           Sweden| 2880|
|           Poland| 2744|
|      Netherlands| 2488|
|            Spain| 2310|
|           Taiwan| 2128|
|            India| 1867|
|                 | 1810|
|   Czech Republic| 1507|
+-----------------+-----+
only showing top 20 rows

##### Find out all devices in countries whose batteries need replacements

In [100]:
df.createOrReplaceTempView('df')

sql1 = "select country, count(distinct id) as device_id from df where battery_level == 0 group by country order by device_id desc limit 100"

spark.sql(sql1).show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+---------+
|          country|device_id|
+-----------------+---------+
|    United States|     6858|
|            China|     1415|
|Republic of Korea|     1217|
|            Japan|     1210|
|          Germany|      760|
|   United Kingdom|      650|
|           Canada|      612|
|           Russia|      600|
|           France|      582|
|           Brazil|      374|
|        Australia|      322|
|           Sweden|      293|
|            Italy|      287|
|           Poland|      278|
|      Netherlands|      251|
|            Spain|      223|
|           Taiwan|      207|
|            India|      189|
|                 |      177|
|        Hong Kong|      149|
+-----------------+---------+
only showing top 20 rows

##### Select all countries' devices with high-levels of C02 

In [104]:
sql2 = "select country, round(avg(c02_level),2), count(distinct id) as device_id from df where c02_level > 1400 group by country order by device_id desc limit 100"

spark.sql(sql2).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+------------------------+---------+
|          country|round(avg(c02_level), 2)|device_id|
+-----------------+------------------------+---------+
|    United States|                 1500.52|    17050|
|            China|                 1500.09|     3616|
|Republic of Korea|                 1500.72|     2942|
|            Japan|                 1498.84|     2935|
|          Germany|                 1500.91|     1966|
|   United Kingdom|                 1499.14|     1660|
|           Canada|                 1498.25|     1564|
|           Russia|                 1500.62|     1508|
|           France|                  1500.8|     1353|
|           Brazil|                 1501.04|      856|
|        Australia|                 1500.52|      769|
|           Sweden|                  1500.2|      724|
|            Italy|                 1501.41|      713|
|           Poland|                 1499.58|      664|
|      Netherlands|                 1502.01|      646|
|         

In [ ]:
spark.catalog.dropTempView("df")